In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import community
from networkx.algorithms.community import girvan_newman

In [ ]:
file_path = "connections.txt"
with open(file_path, "r") as file:
    edges = [line.strip().split() for line in file]

In [ ]:
# Create a directed graph
graph = nx.DiGraph()
graph.add_edges_from(edges)

In [ ]:
def dfs(graph, node, discovery_time, low_link, visited, parent, bridges):
    visited[node] = True
    discovery_time[node] = dfs.time
    low_link[node] = dfs.time
    dfs.time += 1

    for neighbor in graph.neighbors(node):
        if not visited[neighbor]:
            dfs(graph, neighbor, discovery_time, low_link, visited, node, bridges)
            low_link[node] = min(low_link[node], low_link[neighbor])

            if low_link[neighbor] > discovery_time[node]:
                bridges.append((node, neighbor))
        elif neighbor != parent:
            low_link[node] = min(low_link[node], discovery_time[neighbor])

In [ ]:
# Tarjan's algorithm
dfs.time = 0
discovery_time = {node: -1 for node in graph.nodes}
low_link = {node: -1 for node in graph.nodes}
visited = {node: False for node in graph.nodes}
bridges = []

for node in graph.nodes:
    if not visited[node]:
        dfs(graph, node, discovery_time, low_link, visited, None, bridges)

print("Bridges:", bridges)

In [ ]:
graph_density = nx.density(graph)
print("Graph Density:", graph_density)

The graph density of 0.0054 indicates, that the graph is sparse, so only a small fraction of possible connections between nodes is present.

In [ ]:
# Calculating degrees for each node
degrees = dict(graph.degree())

max_degree_node = max(degrees, key=degrees.get)
min_degree_node = min(degrees, key=degrees.get)

print("Node with the Highest Degree:", max_degree_node, "Degree:", degrees[max_degree_node])
print("Node with the Lowest Degree:", min_degree_node, "Degree:", degrees[min_degree_node])

In [ ]:
# Calculating in-degrees and out-degrees for each node
in_degrees = dict(graph.in_degree())
out_degrees = dict(graph.out_degree())

max_in_degree_node = max(in_degrees, key=in_degrees.get)
max_out_degree_node = max(out_degrees, key=out_degrees.get)

print("Node with the Highest In-Degree:", max_in_degree_node, "In-Degree:", in_degrees[max_in_degree_node])
print("Node with the Highest Out-Degree:", max_out_degree_node, "Out-Degree:", out_degrees[max_out_degree_node])

In [ ]:
# Calculate centrality measures
closeness_centralities = nx.closeness_centrality(graph)
betweenness_centralities = nx.betweenness_centrality(graph)

eigenvector_centralities = nx.eigenvector_centrality(graph, max_iter=500)

# Finding nodes with the highest closeness, betweenness, and eigenvector centralities
max_closeness_node = max(closeness_centralities, key=closeness_centralities.get)
max_betweenness_node = max(betweenness_centralities, key=betweenness_centralities.get)
max_eigenvector_node = max(eigenvector_centralities, key=eigenvector_centralities.get)

print("Node with the Highest Closeness Centrality:", max_closeness_node, "Closeness Centrality:", closeness_centralities[max_closeness_node])
print("Node with the Highest Betweenness Centrality:", max_betweenness_node, "Betweenness Centrality:", betweenness_centralities[max_betweenness_node])
print("Node with the Highest Eigenvector Centrality:", max_eigenvector_node, "Eigenvector Centrality:", eigenvector_centralities[max_eigenvector_node])


#### Closeness Centrality:

Node 2642 has the highest closeness centrality with a value of 0.117975. Node 2642 is relatively central in terms of closeness, indicating that it can efficiently reach other nodes in the network.

#### Betweenness Centrality:

Node 1684 has the highest betweenness centrality with a value of 0.032999. Node 1684 plays a crucial role in connecting different parts of the network, serving as a bridge in the flow of information.

#### Eigenvector Centrality:

Node 2655 has the highest eigenvector centrality with a value of 0.802704. Node 2655 is considered highly central in terms of eigenvector centrality, suggesting that it is connected to other nodes that are themselves well-connected.

In [ ]:
undirected_graph = graph.to_undirected()

# Performing community detection using the Louvain method
partition = community.best_partition(undirected_graph)
num_communities = max(partition.values()) + 1

print("Number of Communities:", num_communities)

In [ ]:
# Calculating the sizes of each community
community_sizes = {community_id: sum(1 for node in partition if partition[node] == community_id) for community_id in set(partition.values())}

largest_community_id = max(community_sizes, key=community_sizes.get)
smallest_community_id = min(community_sizes, key=community_sizes.get)

print("Size of the Largest Community (Community ID:", largest_community_id, "):", community_sizes[largest_community_id])
print("Size of the Smallest Community (Community ID:", smallest_community_id, "):", community_sizes[smallest_community_id])

ID 5 is the Largest Community

ID 3 is the Smallest Community

In [ ]:
# Finding the largest three communities
largest_communities = sorted(community_sizes, key=community_sizes.get, reverse=True)[:3]

plt.figure(figsize=(15, 5))
for i, community_id in enumerate(largest_communities, 1):
    plt.subplot(1, 3, i)
    
    community_nodes = [node for node in graph.nodes() if partition[node] == community_id]
    subgraph = graph.subgraph(community_nodes)
    
    pos = nx.spring_layout(subgraph)
    nx.draw(subgraph, pos, with_labels=True, font_weight='bold', node_size=700, node_color="skyblue", arrowsize=15)
    plt.title(f"Community {community_id}")

plt.tight_layout()
plt.show()

In [ ]:
centrality_measures = ['degree_centrality', 'closeness_centrality', 'betweenness_centrality']

for measure in centrality_measures:
    plt.figure(figsize=(15, 5))
    
    for i, community_id in enumerate(largest_communities, 1):
        plt.subplot(1, 3, i)
        
        if measure == 'degree_centrality':
            centrality_values = dict(nx.degree_centrality(graph.subgraph(community_nodes)))
        elif measure == 'closeness_centrality':
            centrality_values = dict(nx.closeness_centrality(graph.subgraph(community_nodes)))
        elif measure == 'betweenness_centrality':
            centrality_values = dict(nx.betweenness_centrality(graph.subgraph(community_nodes)))
        
        # Removing the top 3 nodes based on centrality measure
        top_nodes = sorted(centrality_values, key=centrality_values.get, reverse=True)[:3]
        graph.remove_nodes_from(top_nodes)
        
        pos = nx.spring_layout(graph)
        nx.draw(graph, pos, with_labels=True, font_weight='bold', node_size=700, node_color="skyblue", arrowsize=15)
        plt.title(f"Top 3 Nodes Removed ({measure.capitalize()}) in Community {community_id}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
influencers = {}
centrality_measures = ['degree_centrality', 'closeness_centrality', 'betweenness_centrality']

for measure in centrality_measures:
    influencers[measure] = {}
    
    for community_id in largest_communities:
        community_nodes = [node for node in graph.nodes() if partition[node] == community_id]
        
        # Calculate centrality for each node in the community
        if measure == 'degree_centrality':
            centrality_values = dict(nx.degree_centrality(graph.subgraph(community_nodes)))
        elif measure == 'closeness_centrality':
            centrality_values = dict(nx.closeness_centrality(graph.subgraph(community_nodes)))
        elif measure == 'betweenness_centrality':
            centrality_values = dict(nx.betweenness_centrality(graph.subgraph(community_nodes)))
        
        # Identify the node with the highest centrality in the community
        top_node = max(centrality_values, key=centrality_values.get)
        influencers[measure][community_id] = top_node

# Draw the graph highlighting the influencers
plt.figure(figsize=(15, 7))

for i, measure in enumerate(centrality_measures, 1):
    plt.subplot(1, 3, i)
    pos = nx.spring_layout(graph)
    nx.draw(graph, pos, with_labels=True, font_weight='bold', node_size=700, node_color="skyblue", arrowsize=15)

    for community_id, influencer in influencers[measure].items():
        nx.draw_networkx_nodes(graph, pos, nodelist=[influencer], node_color="red", node_size=700)

    plt.title(f"Top Influencers ({measure.capitalize()})")

plt.tight_layout()
plt.show()

## Action Plan for Targeted Marketing Campaign

**a) Hypothetical Business:**

Let's consider "SkyNet Mobile", a growing telecom company aiming to increase market share and brand awareness.

**b) Hypothetical Marketing Budget:**

We'll set a budget of $1 million for the campaign, spread across a 3-month period.

**c) Hypothetical Cost per Action (CPA):**

- Facebook Ads: $0.50 per click
- Instagram Ads: $1.00 per click
- YouTube Ads: $0.20 per view
- Influencer Marketing: $5,000 per post (estimated)

**d) Target Audience:**

Based on the hypothetical scenario and available budget, here are potential target segments:

**1. Tech-Savvy Young Professionals:**

* **Age:** 25-35
* **Interests:** Technology, travel, social media, entertainment
* **Channels:** Facebook Ads, Instagram Ads, YouTube Ads (Tech reviews, lifestyle content)
* **Value Proposition:** Competitive data plans, bundled entertainment offers, innovative features

**2. Value-Conscious Families:**

* **Age:** 35-55
* **Interests:** Family activities, affordability, reliability
* **Channels:** Facebook Ads, TV Ads, Flyers in local stores
* **Value Proposition:** Affordable family plans, parental control features, reliable network coverage

**3. Rural Residents:**

* **Age:** 35+
* **Interests:** Affordable communication, local news, community
* **Channels:** Radio Ads, Local Events Sponsorship, Mobile App Promotion
* **Value Proposition:** Improved connectivity in rural areas, affordable call rates, community building initiatives

**Campaign Strategies:**

* **Personalized Messaging:** Tailor messages to each segment's interests and pain points.
* **Multi-Channel Approach:** Utilize different channels to reach the target audience effectively.
* **Content Marketing:** Create informative and engaging content relevant to each segment.
* **Data-Driven Optimization:** Monitor campaign performance and adjust strategies based on results.

**Metrics for Success:**

* **Increase in website traffic and app downloads**
* **Growth in social media engagement**
* **Brand awareness surveys**
* **New customer acquisition and retention rates**

**Decision on Whom to Target:**

The final decision on whom to target will depend on several factors, including:

* **Budget:** Reaching a broader audience will require a higher budget.
* **Competition:** Targeting a less saturated segment might be easier to achieve success.
* **Potential ROI:** Focus on segments with higher potential return on investment.
* **Long-Term Goals:** Align target segments with SkyNet Mobile's long-term growth strategy.